In [ ]:
import matplotlib.pyplot as plt
import os
import h5py
import numpy as np

os.environ['HDF5_USE_FILE_LOCKING'] = 'TRUE'

def GetSize(path):
    file = h5py.File(path, mode='r')
    n = file["times"].shape[0]
    file.close()
    return n

def Load(path, chunk=[]):
    # @brief Reads grid data and time data from a specified hdf5 file.
    # @param path The path of the HDF5 file
    # @param dataset A string name of the main dataset containing the states data
    # @param times A string name of the time dataset containing all time points
    # @param chunk An optional [start, end] list that specifies how many time steps to load
    file = h5py.File(path, "r")
    if chunk:
        data_array = np.array(file['data'][chunk[0]:chunk[1],0,:,:])
        time_array = np.array(file['times'][chunk[0]:chunk[1]])
    else:
        data_array = np.array(file['data'][:, 0, :, :])
        time_array = np.array(file['times'])
    file.close()
    return data_array #, time_array)

path = '/srv/public/Divider_c24test.h5'

file = h5py.File(path, mode='r')
data = np.array(file['data'][:, 0, :, :])
alpha = np.array(file['data'][:, -1, :, :])
times = np.array(file['times'])
dx = file['data'].attrs['cell_size']
xlower = file['data'].attrs['xlower']
file.close()

nx = [data.shape[1], data.shape[2]]
xupper = xlower + dx * nx

vmin = None #0.4
vmax = None #2.0

ns = [i for i in range(data.shape[0]) if i % 5 == 0]
for n in ns:
    rho = data[n,:,:]
    nticks = 5

    real_x = np.around(np.linspace(xlower[0], xupper[0], nticks, endpoint=True), 3)
    real_y = np.around(np.linspace(xlower[1], xupper[1], nticks, endpoint=True), 3)

    dxt = [int(nx[0] / (nticks - 1)), int(nx[1] / (nticks - 1))]
    xticks = list(range(0, nx[0] + 1, dxt[0]))
    yticks = list(range(0, nx[1] + 1, dxt[1]))
    
    volfrac = alpha[n, :, :]
    imdata = np.reshape(rho, [nx[1], nx[0]])
    volfrac = np.reshape(volfrac, [nx[1], nx[0]])# np.log(1.0 + np.reshape(rho, [nx[1], nx[0]]))
    imdata = np.where(volfrac == 0.0, np.nan, imdata)
    f, ax = plt.subplots(figsize=(10, 7))
    if vmin is not None:
        im = ax.imshow(imdata, origin='lower', vmin=vmin, vmax=vmax, interpolation='none')
    else:
        im = ax.imshow(imdata, origin='lower', cmap='jet', interpolation='none')
    plt.colorbar(im)
    ax.set_xticks(xticks)
    ax.set_xticklabels(real_x)
    ax.set_yticks(yticks)
    ax.set_yticklabels(real_y)
    ax.set_title('Time: {:.3}s'.format(times[n]))
    plt.show()

In [ ]:
workdir_path = '/srv/public/Maikel/FiniteVolumeSolver/extra/'

import sys
sys.path.append(workdir_path)

import yt
import os
import numpy as np
import matplotlib.pyplot as plt
import amrex.yt_io as da

base_data_path = '/srv/public/Divider_DE5_2400'
plotfiles_path = '{}/Plotfiles'.format(base_data_path)
dirs = os.listdir(plotfiles_path)
plotfiles = ['{}/{}'.format(plotfiles_path, plt) for plt in dirs]
list.sort(plotfiles)

figure_out_path = '{}/Divider/Visualization'.format(workdir_path)
os.makedirs(figure_out_path, exist_ok=True)

for i, plotfile in enumerate(plotfiles):
   (p, rho, rhou, rhov, c, vols), current_time, extent = da.yt_load(plotfile, ["Pressure", "Density", "Momentum_0", "Momentum_1", "SpeedOfSound", 'vfrac'])
   print(p)
   print('{}: {}'.format(plotfile, current_time))
   f, axs = plt.subplots(nrows=3, ncols=1, figsize=(10, 30))
   f.suptitle('Time = {:.2f}'.format(float(current_time)))
   # pressure image
   im_p = axs[0].imshow(p, origin='lower', vmin=1.0e4, vmax=2e5, interpolation='none', extent=extent, cmap='tab20c')
   axs[0].set_title('Pressure')
   plt.colorbar(im_p, ax=axs[0])
   # temperature image
   im_T = axs[1].imshow(rho, origin='lower', vmin=0.0, vmax=3.0, interpolation='none', extent=extent)
   axs[1].set_title('Temperature')
   plt.colorbar(im_T, ax=axs[1])
   # velocity field
   u = np.where(vols > 1e-14, rhou / rho, np.nan)
   v = np.where(vols > 1e-14, rhov / rho, np.nan)
   skip = 50
   scale = 0.2
   axs[2].quiver(u[::skip,::skip], v[::skip,::skip], scale=scale, units='xy')
   axs[2].set_title('Velocity Field')
   f.savefig('{}/Figure{:04d}.png'.format(figure_out_path, i))
   #f.clear()
   #plt.close(f)